# Vault Setup
This notebook has three ways of starting up Vault.

1. A Dockerized Vault OSS
1. A Dockerized Vault Enterprise version
1. A local version of Vault

In [ ]:
# Set Env
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)
# Set dir env vars. Create required directory.
MAIN_DIR=$(pwd)
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}

## Container
This sets up Vault in a Docker container. Run either this or the localhost version below. 

In [ ]:
# (optional) Clean up any running containers
docker stop vault; docker rm vault

In [ ]:
#==> Pull docker images
# docker pull hashicorp/vault-enterprise:latest
# docker pull hashicorp/vault:latest
docker pull hashicorp/vault-enterprise:1.7.3_ent # 1.7.x has 8 hour eval
docker pull hashicorp/vault:1.7.3 # 1.7.x has 8 hour eval

### Vault Container Configuration

There are two option to choose below.

* OSS vs Enterprise
  * Vault OSS
	* comment out this line `image: hashicorp/vault-enterprise`
	* uncomment this line `image: hashicorp/vault`
  * Vault Enterprise
	* uncomment this line `image: hashicorp/vault-enterprise`
	* comment out this line `image: hashicorp/vault`
* Dev Mode vs local config (requires initialization)
  * Dev mode
	* uncomment this line `entrypoint: "vault server -dev`
    * comment out this line `entrypoint: "vault server -config=/vault/config"`
  * Local config
	* vice versa

The default is to run Vault Enterprise in Dev Mode.

Create a `docker-compose.yaml` file.

In [ ]:
## Create docker-compose file.
cat << EOF > docker-compose.yaml
version: '3.7'
services:
  vault:
    image: hashicorp/vault-enterprise:1.7.3_ent # Vault Enterprise
    #image: hashicorp/vault:1.7.3 # Vault OSS
    container_name: vault
    restart: always
    volumes:
#       - ./config/vault/data:/vault/data
      - ./config/vault/logs:/vault/logs
      - ./config/vault/config:/vault/config:ro
#     networks: 
#       - hashi
    ports:
      - "8200:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: ${VAULT_TOKEN}
      VAULT_DEV_LISTEN_ADDRESS: '0.0.0.0:8200'
      VAULT_ADDR: ${VAULT_ADDR}
    cap_add:
      - IPC_LOCK
    #entrypoint: "vault server -dev" # dev mode
    entrypoint: "vault server -config=/vault/config" # non-dev with local config

# networks:
#  hashi:
#    driver: bridge
EOF

### Vault Configuration - non-Dev Mode (WIP)

If want to run with a local config, then create a Vault config file. This file is placed in a folder mounted by the container.

In [ ]:
cat << EOF > $MAIN_DIR/$WORK_DIR/config/vault1.hcl
# storage "file" {
#     path = "/vault/file"
# }

storage "raft" {
  path    = "./vault/data"
  node_id = "node1"
}

listener "tcp" {
  address       = "0.0.0.0:8200"
#   tls_cert_file = "/path/to/fullchain.pem"
#   tls_key_file  = "/path/to/privkey.pem"
  tls_disable = true
}

default_lease_ttl = "168h" # default(768h)
max_lease_ttl = "0h" # default(768h)
api_addr = "http://0.0.0.0:8200"
cluster_addr = "https://0.0.0.1:8201"
ui = true

# seal "pkcs11" {
#   lib            = "/usr/vault/lib/libCryptoki2_64.so"
#   slot           = "0"
#   pin            = "AAAA-BBBB-CCCC-DDDD"
#   key_label      = "vault-hsm-key"
#   hmac_key_label = "vault-hsm-hmac-key"
# }

log_level = debug
telemetry {
  prometheus_retention_time = "30s"
  disable_hostname = true
}
EOF

### Start Container(s)

In [ ]:
docker compose up -d --quiet-pull

## localhost
This sets up Vault on the local host. Run either this or the Docker version above. 

In [ ]:
#Set Env
export VAULT_PORT=8201
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)

In [ ]:
vault server \
  -dev -dev-root-token-id=${VAULT_TOKEN} -dev-listen-address=:8201 \
  -log-level=trace \
  -config=./config.hcl > vault.log 2>&1 &

## Vault - Initialize and Unseal

Only for non-Dev mode, you will need to initialize and unseal your cluster.

In [ ]:
vault operator init -key-shares=1 -key-threshold=1 > $MAIN_DIR/$WORK_DIR/vault.init

In [ ]:
VAULT_TOKEN=$(grep "Root Token" $MAIN_DIR/$WORK_DIR/vault.init | awk '{print $NF}')
UNSEAL_KEY1=$(grep "Key 1" $MAIN_DIR/$WORK_DIR/vault.init | awk '{print $NF}')

In [ ]:
vault operator unseal ${UNSEAL_KEY1}

## Status, Audit, and License

In [ ]:
vault status

In [ ]:
# Enable audit output to file, audit settings are in vault1.hcl
vault audit enable file file_path=/vault/logs/audit.log

In [ ]:
#==> Install License for Vault Enterprise 1.7.x.
## v1.8.x requires Vault to check for license file before starting
## If setting up Vault Enterprise, place license key file in ../../license/vault.hclic
# sleep 5
curl \
    --silent \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --request PUT \
    --data '{"text":"'$VAULT_LICENSE'"}' \
    $VAULT_ADDR/v1/sys/license

## UI

In [ ]:
open $VAULT_ADDR
echo "login with token: $VAULT_TOKEN"

## Sample Authentication Methods

* For LDAP configuration, see [LDAP](./110-Setup-authmethods.ipynb#LDAP)
* For User Pass, see [UserPass](./110-Setup-authmethods.ipynb#UserPass)

## Clean Up

### Clean up Docker

In [ ]:
# docker stop vault-enterprise vault
# docker rm vault-enterprise vault
docker compose down
docker ps -a

### Clean up localhost

In [ ]:
pkill vault

### Clean up artifacts

In [ ]:
rm -rf config/vault